In [1]:
import json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
import numpy as np
import tensorflow as tf

import sentencepiece as spm

# Below code prevents pre-allocating whole available GPU memory.
physical_devices = tf.config.list_physical_devices('GPU')
try:
    for idx in range(len(physical_devices)):
        tf.config.experimental.set_memory_growth(physical_devices[idx], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

from utils_model import MODEL

In [2]:
with open('config_large.json', 'r') as openfile:
    config = json.load(openfile)
print(config)

SOURCE_LEN = config['SOURCE_LEN']
TARGET_LEN = config['TARGET_LEN']
VOCAB_SIZE = config['VOCAB_SIZE']
N = config['N']
H = config['H']
D_MODEL = config['D_MODEL']
DROPOUT = config['DROPOUT']
D_FF = D_MODEL * 4
D_KV = D_MODEL // H # This is default in PyTorch MHA and cannot be set explicitly.

{'N': 6, 'H': 12, 'D_MODEL': 768, 'DROPOUT': 0, 'SOURCE_LEN': 256, 'TARGET_LEN': 256, 'VOCAB_SIZE': 256}


In [3]:
# Tokenizer
tokenizer = spm.SentencePieceProcessor('char_tokenizer/char_tokenizer.model')

In [4]:
# Transformer Model
model = MODEL(N = N, SOURCE_LEN = SOURCE_LEN, TARGET_LEN = TARGET_LEN,
              VOCAB_SIZE = VOCAB_SIZE, D_MODEL = D_MODEL, D_FF = D_FF, D_KV = D_KV,
              H = H, DROPOUT = DROPOUT)
# Dummy forward pass to initiate the weights before loading
X_enc = np.random.randint(low = 0, high = VOCAB_SIZE, size = (1, SOURCE_LEN))
X_dec = np.random.randint(low = 0, high = VOCAB_SIZE, size = (1, TARGET_LEN))
_ = model((X_enc, X_dec))

model.load_weights(f'checkpoints_large/checkpoint_epoch_0010.hdf5')

In [5]:
def infer(text):
    tokenized = tokenizer.encode_as_ids(text)
    encoder_input = tf.keras.preprocessing.sequence.pad_sequences([tokenized], maxlen = SOURCE_LEN, 
                                                            padding = 'post', truncating = 'post')
    decoder_input = [2]
    decoder_input = tf.keras.preprocessing.sequence.pad_sequences([decoder_input], maxlen = TARGET_LEN, 
                                                            padding = 'post', truncating = 'post')
    for idx in range(TARGET_LEN - 1):
        logits = model((encoder_input, decoder_input))
        predicted_token = tf.argmax(logits, axis = -1).numpy()[0][idx]
        if predicted_token == 3:
            break
        decoder_input[0, idx+1] = predicted_token
        
    arr = decoder_input[0]
    return tokenizer.decode_ids(arr[arr>3].tolist())

In [6]:
# You can write your test cases here:
test_cases = ["merhaba benim adım melikşah türker 30 yaşındayım istanbulda yaşıyorumm",
              "boğaziçi üniversitesinde doktora yapıyrum",
              "kumru hanim adında 4 yasinda bi kedi kizim var"]

In [7]:
for case in test_cases:
    print(infer(case))

Merhaba benim adım Melikşah Türker 30 yaşındayım, İstanbul'da yaşıyorum.
Boğaziçi Üniversitesi'nde doktora yapıyrum.
Kumru Hanim adında 4 yasinda bi kedi kizim var.
